In [1]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from Login_info import user as user1, password as pass1
import time, math
import pandas as pd

In [2]:
import linkedin_scraping as ls

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome('chromedriver',options=options)

In [4]:
time.sleep(3)
driver.get("https://www.linkedin.com")

time.sleep(1)

In [5]:
ls.Login(driver,user1,pass1)

In [15]:
keyword = 'Applaudo Studios'  
posts = ls.Posts_pagination(driver, keyword)

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20
Page: 21
Page: 22
Page: 23
Page: 24
Page: 25
Page: 26
Page: 27
Page: 28
Page: 29
Page: 30
Page: 31
Page: 32
Page: 33
Page: 34
Page: 35
Page: 36
Page: 37
Page: 38
Page: 39


In [16]:
posts_df = pd.DataFrame(posts, columns=['post_url'])
posts_df.to_csv('data/'+keyword+' posts_url.csv', index=False) 

In [17]:
posts_url_df = pd.read_csv('data/'+keyword+' posts_url.csv')
posts_url_df.head()

,post_url
0,https://www.linkedin.com/feed/update/urn:li:ac...
1,https://www.linkedin.com/feed/update/urn:li:ac...
2,https://www.linkedin.com/feed/update/urn:li:ac...
3,https://www.linkedin.com/feed/update/urn:li:ac...
4,https://www.linkedin.com/feed/update/urn:li:ac...


In [21]:
def get_media(wd, post_elmt):
    media_url = ''
    media_type = 'Other'  # unknown o no media?

    try:
        media_url = post_elmt.find_element_by_class_name(
            'vjs-tech').get_attribute('src')
        media_type = 'Video'
    except:
        try:
            media_url = post_elmt.find_element_by_class_name(
                'tap-target.feed-shared-mini-update-v2__link-to-details-page.text-body-medium.ember-view').get_attribute('href')
            media_type = 'Shared LinkedIn post'
        except:
            try:
                media_url = post_elmt.find_element_by_class_name(
                    'app-aware-link.feed-shared-article__meta.flex-grow-1.full-width.tap-target').get_attribute('href')
                media_type = 'Shared link outside LinkedIn without image'
            except:
                try:
                    media_url = post_elmt.find_element_by_class_name(
                        'app-aware-link.feed-shared-article__image-link.tap-target').get_attribute('href')
                    media_type = 'Shared link outside LinkedIn'  # job or article or whatever
                except:
                    try:
                        media_url = post_elmt.find_element_by_class_name(
                            'app-aware-link.feed-shared-entity__content.display-flex').get_attribute('href')
                        media_type = 'Job offer posted on LinkedIn'
                    except:
                        try:
                            media_url = post_elmt.find_element_by_class_name(
                                'app-aware-link.feed-shared-article__meta.flex-grow-1.full-width.tap-target').get_attribute('href')
                            media_type = 'Shared link outside LinkedIn without image'
                        except:
                            try:
                                media_url = post_elmt.find_element_by_class_name(
                                    'app-aware-link.tap-target.video-s-loader__play-link.play-video').get_attribute('href')
                                media_type = 'External video'
                            except:
                                images = post_elmt.find_elements_by_class_name(
                                    'ivm-view-attr__img--centered.feed-shared-image__image.lazy-image.ember-view')
                                if len(images) == 1:
                                    media_url = images[0].get_attribute('src')
                                    media_type = 'Single image'
                                elif len(images) > 1:
                                    media_url = []
                                    for img in images:
                                        media_url.append(
                                            img.get_attribute('src'))
                                    media_type = 'Multiple images'
                                else:
                                    wd.switch_to.frame(
                                        wd.find_element_by_tag_name("iframe"))
                                    carousel = wd.find_elements_by_class_name(
                                        'carousel-lazy-element')
                                    if len(carousel) > 0:
                                        media_url = []
                                        for slide in carousel:
                                            media_url.append(
                                                slide.get_attribute('src'))
                                        media_type = 'Carousel'
                                        wd.switch_to.default_content()
                                    else:
                                        wd.switch_to.default_content()

    return media_url, media_type


def get_post(wd):
    post_df = pd.DataFrame(
        columns=['post_url', 'author', 'author_url', 'author_type', 'description', 'media_type', 'media_url'])

    author_element_cn = 'app-aware-link.feed-shared-actor__container-link.relative.display-flex.flex-grow-1'
    author_name_cn = 'feed-shared-actor__name.t-14.t-bold.hoverable-link-text.t-black'
    post_description_cn = 'feed-shared-inline-show-more-text.feed-shared-update-v2__description.feed-shared-inline-show-more-text--minimal-padding.feed-shared-inline-show-more-text--expanded'

    post_element = wd.find_element_by_class_name('core-rail.update-outlet')
    author_url = post_element.find_element_by_class_name(
        author_element_cn).get_attribute('href').split('?')[0]
    author_name = post_element.find_element_by_class_name(author_name_cn).text
    author_type = 'Company' if author_url.find('company') != -1 else 'User'

    try:
        description = post_element.find_element_by_class_name(
            post_description_cn).text
    except:
        description = 'No description'

    try:
        source = post_element.get_attribute('src')
    except:
        source = 'No source'

    media_url, media_type = get_media(wd, post_element)

    post_df = post_df.append({'post_url': wd.current_url, 'author': author_name, 'author_url': author_url,
                              'author_type': author_type, 'description': description, 'media_type': media_type,
                              'media_url': media_url},
                             ignore_index=True)
    print('media_url', media_url)
    print('media_type', media_type)

    return post_df

In [37]:
driver.switch_to.default_content()
post = ls.get_post(driver)
post

# Problemas con los carousels

media_url ['https://media-exp1.licdn.com/dms/image/C4E1FAQHmvsKrTIJ2zQ/feedshare-document-images_480/1/1617142677880?e=1625958000&v=beta&t=rwCrnStpvg6wd2fBYe0rFknggaAxmkpYDPkIrQQJKsg', 'https://media-exp1.licdn.com/dms/image/C4E1FAQHmvsKrTIJ2zQ/feedshare-document-images_160/2/1617142677839?e=1625958000&v=beta&t=B-zu_SEru52JhAKpQQ90aRWh5015XMjSnsRP_FGbLFQ', None]
media_type Carousel


,post_url,author,author_url,author_type,description,media_type,media_url
0,https://www.linkedin.com/feed/update/urn:li:ac...,Applaudo Studios,https://www.linkedin.com/company/applaudo-stud...,Company,Take a look at how machine learning can benefi...,Carousel,[https://media-exp1.licdn.com/dms/image/C4E1FA...


In [67]:
# driver.switch_to.default_content()
# driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
# driver.find_elements_by_class_name('carousel-lazy-element')
reactions_window_btn = 'v-align-middle.social-details-social-counts__reactions-count'
try:
    reactions = driver.find_element_by_class_name(reactions_window_btn)
except:
    reactions_dataframe = reactions_dataframe.append({'post_url': wd.current_url,
                                                          'username': '', 'profile_url': '', 'reaction': ''},
                                                         ignore_index=True)
#     return reactions_dataframe

reactions_window_class_name = 'artdeco-modal__content.social-details-reactors-modal__content.ember-view'
panel = driver.find_element_by_class_name(reactions_window_class_name)
n_reactions = int(reactions.text)
for i in range(int(n_reactions/10)+1):
        driver.execute_script(
            'arguments[0].scrollTop = arguments[0].scrollHeight', panel)
        time.sleep(2)
        try:
            driver.find_element_by_xpath("//*[text()='Show more results']").click()
        except:
            pass

In [68]:
posts_reactions = pd.DataFrame()
posts_comments = pd.DataFrame()
posts = pd.DataFrame()

# open chrome console

for i, row in posts_url_df.iterrows():
    if i > 4:
        break
    driver.get(row['post_url'])
    time.sleep(2)
    post = ls.get_post(driver)
    posts = pd.concat([posts, post])
    post_r = ls.get_post_reactions(driver)
    posts_reactions = pd.concat([posts_reactions,post_r]) 
    post_c = ls.get_post_comments(driver)
    posts_comments = pd.concat([posts_comments,post_c])
    print(i)
    

media_url ['https://media-exp3.licdn.com/dms/image/C4D1FAQEfNOb1DM4-Gg/feedshare-document-images_480/1/1625675311597?e=1625950800&v=beta&t=W90MSHYlDviOyL8EGP7_rLNmua5YMuMoCQEhCh9zZKA', 'https://media-exp3.licdn.com/dms/image/C4D1FAQEfNOb1DM4-Gg/feedshare-document-images_160/2/1625675311541?e=1625950800&v=beta&t=pb3SId22RSUTH1Q2MmEbU_Mk3XL6CUAgoqqZI0GG0_k', None]
media_type Carousel


KeyboardInterrupt: 

In [ ]:
posts.tail()

In [ ]:
posts_comments.tail()

In [ ]:
reactions_window_btn = 'v-align-middle.social-details-social-counts__reactions-count'
driver.find_element_by_class_name(reactions_window_btn)

In [ ]:
# for commentor in wd.find_elements_by_class_name(commentors_name_class_name):
    #     commentors_name.append(commentor.text)
    
    
    183

In [ ]:
posts_comments.head()

In [ ]:


element2 = driver.find_element_by_xpath("//div[@class='social-details-reactors-tab-body']")
elements3 = element2.find_elements_by_xpath(".//div[@class='artdeco-entity-lockup__title ember-view']")

for element in elements3:
    print(element.text)
    

In [ ]:
# 'https://www.linkedin.com/posts/anaromerolinkepower_equipopower-familiaempresarial-activity-6817819334484774912-B4R2/'
# https://www.linkedin.com/posts/jorgebranger_nunca-dejes-pasar-una-oportunidad-por-estar-activity-6815238905818832896-ix4P/

post_comments = get_post_comments(driver)
post_comments.head()

In [ ]:
post_reactions_and_comments = pd.concat([post_reactions,post_comments])
post_reactions_and_comments.to_csv('C:/Users/VanessaNunez/Downloads')

In [ ]:
# fixed full
# feed-shared-update-v2__description-wrapper

In [ ]:
for k in reactors.keys():
    print(k)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
driver.find_elements_by_class_name('social-details-reactors-tab-body')

In [ ]:
#driver.close()

In [ ]:
driver.window_handles

In [ ]:
driver.current_window_handle

In [ ]:
def Posts(wd):
    time.sleep(2)
    # Click in Posts
    posts_url = wd.current_url + "posts/"
    wd.get(posts_url)
    time.sleep(2)
    
    # Scroll till end of page
    inicial_yoffset = 0
    scrolled_yoffset = driver.execute_script("return window.pageYOffset;") + 1 # Inicial return is 0

    while scrolled_yoffset > inicial_yoffset:
        inicial_yoffset = driver.execute_script("return window.pageYOffset;")
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(1)
        scrolled_yoffset = driver.execute_script("return window.pageYOffset;")
        
    # String to look
    data_urn = 'urn:li:activity:'
    posts = {}
            
    # Get posts url
    for i in wd.find_elements_by_class_name("occludable-update.ember-view"):
        posts[i.get_attribute('href')] = 1

    # Click next page
    if j + 1 < n_pages:
        wd.find_elements_by_class_name('artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view')[0].click()

    return posts
        